In [10]:
import json

class fubot_config():
    class feat_group():
        def __init__(self, feature_config):
            self.name = None
            self.bone_name = None
            self.elements = []
            self.__parse__(feature_config)

        def __parse__(self, feature_config):
            self.name = feature_config[0]
            self.bone_name = feature_config[1] if len(feature_config[1]) > 0 else self.name
            self.elements = feature_config[2]

    def __init__(self, config_path):
        self.features_output:feat_group = []
        self.__loadConfig__(config_path)

    def __loadConfig__(self, config_path):
        with open(config_path) as f:
            config_json = json.loads(f.read())

        #output features
        features = config_json['training']['output_features']
        for feat in features:
            self.features_output.append(self.feat_group(feat))

config = fubot_config('../samples/generator_config.json')

In [16]:
from collections import OrderedDict
from bvhtoolbox import Bvh
import transforms3d as t3d

class bvh_skeleton_node:
    def __init__(self, name, offset):
        self.children = []
        self.parent:bvh_skeleton_node = None

        self.name = name
        self.local_position = offset
        self.local_rotation_mat = None
        self.local_rotation_quat = [0,0,0,0]
        self.local_transform = None

        self.world_position = None
        self.world_rotation_mat = None
        self.world_rotation_quat = None
        self.world_transform = None

    def CalculateWorld(self):
        self.local_transform = t3d.affines.compose(self.local_position, self.local_rotation_mat, [1,1,1])

        if self.parent != None:
            self.world_transform = self.parent.world_transform.dot(self.local_transform)
        else:
            self.world_transform = self.local_transform

        self.world_position, self.world_rotation_mat, _, _ = t3d.affines.decompose(self.world_transform)
        self.world_rotation_quat = t3d.quaternions.mat2quat(self.world_rotation_mat)

        for child in self.children:
            child.CalculateWorld()

    def SetLocalRotation(self, new_rotation):
        self.local_rotation_quat = new_rotation
        self.UpdateLocalRotation()

    def UpdateLocalRotation(self):
        self.local_rotation_mat = t3d.quaternions.quat2mat(self.local_rotation_quat)

    def AddChild(self, child):
        self.children.append(child)
        child.parent = self
        return child #Enables chaining

    def print_tree(self):
        if self.parent is None:
            print(f'{self.name} : ROOT ')
        else:
            print(f'{self.name} : {self.parent.name}')

        print(f'\t{self.name}_world {{{self.world_position}, {self.world_rotation_quat}}}')
        print(f'\t{self.name}_local {{{self.local_position}, {self.local_rotation_quat}}}')

        for child in self.children:
            child.print_tree()

In [17]:
class bvh_skeleton: 
    def __init__(self, bvh_filepath):
        self.root = None
        self.nodes = OrderedDict()
        self.nodes_flat = []
        self.fp_bvh = bvh_filepath
        self.bvh:Bvh = None

        self.__loadSkeleton()

    def __loadSkeleton(self):
        with open(self.fp_bvh) as f:
            self.bvh = Bvh(f.read())
        
        joint_names = self.bvh.get_joints_names() 
        skDef = OrderedDict()
        for joint_name in joint_names:
            j = self.bvh.get_joint(joint_name)
            j_parent = j.parent.value[1] if len(j.parent.value) > 1 else None
            j_offset = [float(val) for val in j['OFFSET']]
            skDef[joint_name] = (j_offset, j_parent)

        #Construct Skeleton
        self.nodes.clear()
        for key, value in skDef.items():
            value[0][0] = -value[0][0]
            new_node = bvh_skeleton_node(key, value[0])
            self.nodes[key] = new_node

            if value[1] != None:
                self.nodes[value[1]].AddChild(new_node)
            else:
                self.root = new_node

        self.nodes_flat = list(self.nodes.values())

    def set_motion_frame(index):
        

    def ApplyMotion(self, frameData):
        #Root Local Position
        fQ = deque(frameData)
        self.root.local_position[0] = fQ.popleft()
        self.root.local_position[1] = fQ.popleft()
        self.root.local_position[2] = fQ.popleft()

        #Joints Local Rotations
        for n in self.nodes_flat:
            n.local_rotation_quat[0] = fQ.popleft()
            n.local_rotation_quat[1] = fQ.popleft()
            n.local_rotation_quat[2] = fQ.popleft()
            n.local_rotation_quat[3] = fQ.popleft()
            n.UpdateLocalRotation()

        self.root.CalculateWorld()

skeleton = bvh_skeleton('../samples/beatsaber_0_1_b.bvh')
skeleton.root.print_tree()

Hips : ROOT 
	Hips_world {None, None}
	Hips_local {[-0.0, 0.0, 0.0], [0, 0, 0, 0]}
LeftUpperLeg : Hips
	LeftUpperLeg_world {None, None}
	LeftUpperLeg_local {[-0.091245, -0.066564, -0.000554], [0, 0, 0, 0]}
LeftLowerLeg : LeftUpperLeg
	LeftLowerLeg_world {None, None}
	LeftLowerLeg_local {[-0.002447, -0.463436, -0.005172], [0, 0, 0, 0]}
LeftFoot : LeftLowerLeg
	LeftFoot_world {None, None}
	LeftFoot_local {[0.002447, -0.42048, -0.020576], [0, 0, 0, 0]}
LeftToes : LeftFoot
	LeftToes_world {None, None}
	LeftToes_local {[-0.003734, -0.104922, 0.126407], [0, 0, 0, 0]}
RightUpperLeg : Hips
	RightUpperLeg_world {None, None}
	RightUpperLeg_local {[0.091245, -0.066564, -0.000554], [0, 0, 0, 0]}
RightLowerLeg : RightUpperLeg
	RightLowerLeg_world {None, None}
	RightLowerLeg_local {[0.002447, -0.463436, -0.005146], [0, 0, 0, 0]}
RightFoot : RightLowerLeg
	RightFoot_world {None, None}
	RightFoot_local {[-0.002447, -0.420479, -0.020602], [0, 0, 0, 0]}
RightToes : RightFoot
	RightToes_world {None, None